<a href="https://colab.research.google.com/github/Dhanush-Poduval/Resume_anylyzer/blob/main/resume_anilyser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install PyPDF2 python-docx spacy wordcloud
!pip install nltk
!python -m spacy download en_core_web_sm
!pip install pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 63.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
from PyPDF2 import PdfReader
import spacy
import re

reader = PdfReader("Resume_Latest.pdf")
text = ""
for i, page in enumerate(reader.pages):
    page_text = page.extract_text()
    text += page_text + "\n"
    print(text)
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)

clean_tokens = [token.text for token in doc if not token.is_stop and not token.is_punct]


programming_languages = [

    'Python', 'Java', 'JavaScript', 'C', 'C++', 'C#', 'Ruby', 'PHP',
    'Swift', 'Kotlin', 'Go', 'TypeScript', 'Dart', 'Rust', 'Scala', 'Objective-C',
    'HTML', 'CSS', 'XML', 'SQL', 'JSON', 'JSX', 'SASS', 'SCSS', 'Less',
    'R', 'MATLAB', 'Julia', 'SAS', 'Bash', 'Shell', 'Perl', 'Lisp', 'Haskell',
    'F#', 'Erlang', 'Ada', 'Fortran', 'COBOL', 'Prolog', 'Scheme',
    'VBA', 'Assembly', 'Solidity', 'Apex', 'Elixir', 'Groovy', 'Crystal', 'Elm'
]
found_languages = [token for token in clean_tokens if token in programming_languages]

percentages = re.findall(r'\b\d{1,3}(?:\.\d+)?\s?%', text)
cgpas = re.findall(r'\b\d\.\d{1,2}/10\b', text)

entities = [(ent.text, ent.label_) for ent in doc.ents if ent.label_ in ['PERSON','ORG','DATE','GPE']]



tech_terms = ['Python','JavaScript','HTML','CSS','ReactJS','NodeJS','Flask','TensorFlow',
              'Data Structures','Design and Analysis of Algorithms']

skills=[token for token in clean_tokens if token in programming_languages]

companies = []
for ent_text, ent_label in entities:
    if ent_label == 'ORG':
        if not any(term in ent_text for term in tech_terms):
            companies.append(ent_text)



resume_data = {
    'Name': [ent[0] for ent in entities if ent[1]=='PERSON'],
    'Companies': companies,
    'Dates': [ent[0] for ent in entities if ent[1]=='DATE'],
    'Languages': found_languages,
    'Percentages': percentages,
    'CGPAs': cgpas
}
print(resume_data)









Dhruv Poduval
/githubhttps://github.com/dhruvpoduval |/linkedinlinkedin.com/in/dhruv-poduval-02aab2261 |/envel⌢pe
dhruvpoduval@gmail.com
Education
PES University 2021-2025
B.Tech Computer Science Current GPA: 8.52/10
National Public School 2019-2021
12th% - 96.8%
Coursework
Courses: Data Structures and Applications, Design and Analysis of Algorithms, Machine Intelligence,
Database Management Systems, Data Analytics, Database Technologies, Object Oriented Design and
Analysis, Generic Programming in C++.
Awards: 2x recipient of the MRD Scholarship for placing in the top 20% at the end of the semester.
Skills
Languages : C/C++(intermediate), Python, JavaScript(basic), HTML/CSS(basic), Java(basic), MySQL, MongoDB
Tools : Git/GitHub, VS Code, Docker
Libraries and Frameworks : ReactJS, NodeJS, Express, Flask, REST API’s
Experience
VuNet Systems |Software Engineering Intern Jan – Aug 2025
•Engineered modular UI automation test suites using Selenium andPlaywright , validating critical workflow

In [18]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shamimhasan8/resume-vs-job-description-matching-dataset")

print("Path to dataset files:", path)
import pandas as pd

import os

path = "/kaggle/input/resume-vs-job-description-matching-dataset"
print(os.listdir(path))

100%|██████████| 927k/927k [00:00<00:00, 69.3MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/shamimhasan8/resume-vs-job-description-matching-dataset/versions/1
['resume_job_matching_dataset.csv']


In [19]:
import pandas as pd
import os

path = "/root/.cache/kagglehub/datasets/shamimhasan8/resume-vs-job-description-matching-dataset/versions/1"
file_path = os.path.join(path, "resume_job_matching_dataset.csv")

df = pd.read_csv(file_path)
print(df.head())
print(df.info())


                                     job_description  \
0  Data Analyst needed with experience in SQL, Ex...   
1  Data Scientist needed with experience in Stati...   
2  Software Engineer needed with experience in Sy...   
3  ML Engineer needed with experience in Python, ...   
4  Software Engineer needed with experience in RE...   

                                              resume  match_score  
0  Experienced professional skilled in SQL, Power...            4  
1  Experienced professional skilled in Python, De...            4  
2  Experienced professional skilled in wait, Git,...            5  
3  Experienced professional skilled in return, De...            4  
4  Experienced professional skilled in REST APIs,...            5  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   job_description  10000 non-null  object
 1   resume  